In [0]:
#We will try to predict what passengers survived the Titanic crash based on features like: age,cabin,children,etc.
df = spark.sql('SELECT * FROM titanic_csv')

In [0]:

df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: float (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [0]:
df.columns

Out[57]: ['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [0]:
#We only select useful columns for us
my_cols=df.select(['Survived','Pclass','Sex','Age', 'SibSp', 'Parch','Fare','Embarked'])

In [0]:
#Now we want to deal with the missing data(null cells)
#We will get rid of them (drop)
my_final_data = my_cols.na.drop()
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer)

In [0]:
#StringIndexer allows us to convert any string to a particular number 
#We will create a gender indexer now
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [0]:
#We are doing the same thing for the embark column
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [0]:
#We now will create an assembler using VectorAssembler
assembler = VectorAssembler(inputCols=['Pclass','SexVec','EmbarkVec','Age','SibSp','Parch','Fare'],outputCol='features')

In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [0]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [0]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,gender_encoder,embark_encoder,assembler,log_reg_titanic])

In [0]:
#Now we do the same things as in a normal model!!
#We will split to train and testa data
train_data,test_data = my_final_data.randomSplit([0.7,0.3])

In [0]:
fit_model = pipeline.fit(train_data)

In [0]:
#We tranform our test data.    We want to test our data on a test data set
results = fit_model.transform(test_data)

In [0]:
#Its time to create evaluate
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [0]:
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [0]:
AUC = my_eval.evaluate(results)

In [0]:
AUC

Out[73]: 0.8000388840283852